In [ ]:
#add mbes_sim to search path
import sys
sys.path.insert(0, "../src/")

In [ ]:
%matplotlib widget

from ipywidgets import *
import matplotlib.pyplot as plt

import numpy as np
import math
from copy import deepcopy
from tqdm.auto import tqdm

import mbes_sim.simulationfunctions as SIMFUN
import mbes_sim.functions.create_bubblesfunctions as bubbles

In [ ]:
# simplify plotting
import matplotlib as mpl

mpl.rcParams['figure.dpi'] = 100
close_plots = True


def create_figure(name):
    if close_plots: plt.close(name)
    fig = plt.figure(name)
    fig.suptitle = name
    
    return fig

In [ ]:
downfactor = 1 #speed up sumlation by reducing the resolution and number of beams and samples of the mbes
resfactor  = 1
voxsize    = 1

beam_pattern = SIMFUN.t_Window.Exponential

method_name = 'sv_int_lin'

In [ ]:
setup = SIMFUN.SimulationSetup(
    addPlots = True,
    prefix = 'examples',
    downfactor = downfactor,
    blockAvg = False,
    resfactor  = resfactor,
    windowType           = beam_pattern,
    idealizedBeampattern = False,
    equiDist             = False,
    motionDataPath = '../test_data/m143_l0154_motion.csv',
    
    surveyType           = SIMFUN.t_Survey.RealMotion,
    voxelsize            = voxsize / downfactor,
    voxelsizeZ           = voxsize / downfactor,
    surveyspeedKnots     = 3,
    swathdistance        = 0.8 / downfactor,
    layerDepths          = [],
    layerSizes           = [],
    bubbleType           = SIMFUN.t_Bubbles.BubbleStream,
    exagHPR              = 1,
    BaseDirectory        = 'GEOMAR_simresults',
    
    load_previous_simresults = False,    
    verbose = True)

BaseSetup = deepcopy(setup.SimSetup)

In [ ]:
sim_setup = setup

sim_survey = sim_setup.get_survey()

sim_survey.setRandomOffsetX(sim_setup.SimSetup['voxelsize'])
sim_survey.setRandomOffsetY(sim_setup.SimSetup['voxelsize'])
sim_survey.setRandomOffsetZ(sim_setup.SimSetup['voxelsize'])
sim_survey.pitchs_degree+=0 # try adding pitch. It is interesting as it bends the bubble stream side lobes (but does not seem to have a negative effect on echo grid integration)
sim_survey.yaws_degree+=0

sim_setup.Simulation.setSurvey(sim_survey)

targets = bubbles.BubbleGenerator().generate_bubblestreams_within_cylindrical_section_along_path(
    start_x = -1.5,
    end_x = 1.5,
    min_range = 0,
    max_range = 130,
    min_beamsteeringangle = -90,
    max_beamsteeringangle = 90,
    min_y = 41,
    max_y = 43,
        zdist_list=[0.1])

if False:
    xbase=targets.x[int(len(targets)/2)]
    ybase=targets.y[int(len(targets)/2)]
    zbase=targets.z[int(len(targets)/2)]
    for i in range(len(targets)):
        targets.x[i] += (targets.z[i] - zbase)/1.1 - xbase
        targets.y[i] += (targets.z[i] - zbase)/1.12 - ybase

sim_setup.Simulation.setTargets(targets)

np.mean(targets.y)

In [ ]:
scatterGrids = sim_setup.Simulation.simulate3DEchoesGrid(progress=True,return_scatterGrids=True)

In [ ]:
#create imshape
scatterGrid = scatterGrids['sv_int_lin']
imshape = scatterGrid.ImageAvg.shape
print(imshape)
print(np.nansum(scatterGrid.TotalValue))
print(targets.sum())

In [ ]:
#find max z respones
max_resp_x = [0,0]
max_resp_y = [0,0]
max_resp_z = [0,0]

for xi in tqdm(range(imshape[0])):
    if False:
        resp_x = np.nanmax(scatterGrid.ImageAvg[xi,:,:])
    else:
        resp_x = np.nansum(scatterGrid.ImageAvg[xi,:,:])
    
    if resp_x > max_resp_x[0]: max_resp_x = [resp_x,xi]
        
for yi in tqdm(range(imshape[1])):
    if False:
        resp_y = np.nanmax(scatterGrid.ImageAvg[:,yi,:])
    else:
        resp_y = np.nansum(scatterGrid.ImageAvg[:,yi,:])
    
    if resp_y > max_resp_y[0]: max_resp_y = [resp_y,yi]
        
for zi in tqdm(range(imshape[2])):
    if False:
        resp_z = np.nanmax(scatterGrid.ImageAvg[:,:,zi])
    else:
        resp_z = np.nansum(scatterGrid.ImageAvg[:,:,zi])
    
    if resp_z > max_resp_z[0]: max_resp_z = [resp_z,zi]
   
print(max_resp_x)
print(max_resp_y)
print(max_resp_z)

In [ ]:
def get_3DImage(self,
             todB= True,
             mindBVal = -100):
    image = self.ImageAvg.copy()
    image[self.ImageNums == 0] = np.nan
    if todB:
        image[image == 0] = 0.000000000000001
        image = 10 * np.log10(image)
        image[image < mindBVal] = mindBVal
        
    
    return image

In [ ]:

def set_layer(ld,ls):
    
    iz1 = ld-ls/2
    iz2 = ld+ls/2
    
    gridder = scatterGrid.getGridder()
    image = get_3DImage(scatterGrid,mindBVal=-100)

    xbox = [gridder.get_x_value(0),gridder.get_x_value(image.shape[0]),gridder.get_x_value(image.shape[0]),gridder.get_x_value(0),gridder.get_x_value(0)]
    ybox = [0,0,0,0,0]
    ybox = [gridder.get_y_value(max_resp_y[1]),gridder.get_y_value(max_resp_y[1]),gridder.get_y_value(max_resp_y[1]),gridder.get_y_value(max_resp_y[1]),gridder.get_y_value(max_resp_y[1])]
    zbox = [gridder.get_z_value(0),gridder.get_z_value(0),gridder.get_z_value(image.shape[2]),gridder.get_z_value(image.shape[2]),gridder.get_z_value(0)]

    z1 = gridder.get_z_grd_value(iz1) - gridder.zres/2
    z2 = gridder.get_z_grd_value(iz2) + gridder.zres/2

    xbox_v12 = [gridder.get_x_value(0),gridder.get_x_value(image.shape[0]),gridder.get_x_value(image.shape[0]),gridder.get_x_value(0),gridder.get_x_value(0)]
    ybox_v12 = [gridder.get_y_value(0),gridder.get_y_value(0),gridder.get_y_value(image.shape[1]),gridder.get_y_value(image.shape[1]),gridder.get_y_value(0)]
    zbox_v1 = [z1,z1,z1,z1,z1]
    zbox_v2 = [z2,z2,z2,z2,z2]

    xbox_v3 = [gridder.get_x_value(0),gridder.get_x_value(0),
               gridder.get_x_value(image.shape[0]),gridder.get_x_value(image.shape[0]),
               gridder.get_x_value(image.shape[0]),gridder.get_x_value(image.shape[0]),
               gridder.get_x_value(0),gridder.get_x_value(0)]
    ybox_v3 = [gridder.get_y_value(0),gridder.get_y_value(0),
                gridder.get_y_value(0),gridder.get_y_value(0),
                gridder.get_y_value(image.shape[1]),gridder.get_y_value(image.shape[1]),
                gridder.get_y_value(image.shape[1]),gridder.get_y_value(image.shape[1])]
    zbox_v3 = [z1,z2,z2,z1,z1,z2,z2,z1]

    xpath = setup.get_survey().pingpositions_x
    ypath = setup.get_survey().pingpositions_y
    zpath = setup.get_survey().heaves

    zi1 = gridder.get_z_index(iz1)
    zi2 = gridder.get_z_index(iz2)
    globals().update(locals())
    
set_layer(108,12)

In [ ]:
cmin=-100
cmap='YlGnBu_r'

mindBVal=-100

In [ ]:
from ipywidgets import *
import numpy as np
import matplotlib.pyplot as plt

name = 'sv_int_lin'
scattergrid = scatterGrids[name]

#get min max
image = scattergrid.ImageAvg.copy()
image[scattergrid.ImageNums == 0] = np.nan
image[image==0] = np.nan
tmp = 10 * np.log10(image)
scattergridCmin = np.nanmin(tmp)
scattergridCmax = np.nanmax(tmp)
print(scattergridCmin,scattergridCmax)

#fig_slices = create_figure('resample - minval[]' + str(cmin) + 'dB] - ' + name)
fig_slices = create_figure('axes sum - ' + name)

defaultcmin=1-abs(cmin/(math.ceil(scattergridCmax)-math.floor(scattergridCmin)))

def update(local_max_db_diff = FloatSlider(min=cmin, max=0, step=1, value=cmin),
           cmin = FloatSlider(min=math.floor(scattergridCmin), max=math.ceil(scattergridCmax), step=1, value=cmin),
           wci_index=(-1,scattergrid.ImageAvg.shape[0]),
           echo_index=(-1,scattergrid.ImageAvg.shape[1]),
           depth_index=(-1,scattergrid.ImageAvg.shape[2]),
           show_wci=True,
           show_echo=False,
           show_map=False,
           local_ld = FloatSlider(min=0, max=125, step=scattergrid.ResZ, value=86),
           local_ls = FloatSlider(min=1, max=125, step=scattergrid.ResZ, value=12)):
    
    global max_db_diff
    max_db_diff = local_max_db_diff
    
    set_layer(local_ld,local_ls)
    cutScatterGrid = scattergrid.cutDepthLayer(ld,ls)
    max_val_db = 10 * math.log10(np.nanmax(cutScatterGrid.ImageAvg))
    
    if wci_index == scattergrid.ImageAvg.shape[0]: wci_index = max_resp_x[1]
    if echo_index == scattergrid.ImageAvg.shape[1]: echo_index = max_resp_y[1]
    if depth_index == scattergrid.ImageAvg.shape[2]: depth_index = max_resp_z[1]
        
    zindeces = None
    if wci_index == -1: wci_index = None
    if echo_index == -1: echo_index = None
    if depth_index == -1: 
        depth_index = None
        zindeces = [zi1,zi2]
    
    global fig_slices
    
    try:
        fig_slices, ax, _ = scattergrid.plot(
            fig_slices,
            targets_color   = [
                (setup.Simulation.Targets,'black'),
            ],
               target_size = 0.01,
            show_wci  = show_wci,
            show_echo = show_echo,
            show_map  = show_map,
            todB      = True,
            mindBVal  = max_val_db+max_db_diff,
            mindBReplace  = cmin,
            xindex = wci_index,
            yindex = echo_index,
            zindex = depth_index,
            zindeces = zindeces,
            #xindex = max_resp_x[1],
            #yindex = max_resp_y[1],
            #zindex = max_resp_z[1],

                kwargs={
                    'cmap' : cmap,
                    #'vmin' : scattergridCmin,
                    #'vmax' : scattergridCmax
                    'vmin' : -100,
                    'vmax' : 0
                }    
        )
    except Exception as e:
        print(e)
        pass
    
    grd = scattergrid.getGridder()
    
    
    if wci_index is not None:
        wci_index_m   = grd.get_x_value(wci_index)    
    if echo_index is not None:
        echo_index_m  = grd.get_y_value(echo_index)
    if depth_index is not None:
        depth_index_m = grd.get_z_value(depth_index)

    axes = iter(fig_slices.get_axes())
    if show_wci:
        ax = next(axes)
        ax.axhline(zi1,c='red')
        ax.axhline(zi2,c='red')
        
    if show_echo:
        ax = next(axes)
        ax.axhline(zi1,c='red')
        ax.axhline(zi2,c='red')
        
    if show_map:
        ax = next(axes)
    
            
    trueval = targets.cutDepthLayer(*cutScatterGrid.ExtentZ).sum()
    print('max:',round(max_val_db,2))
    print('difference from true:',round(100*(1-trueval/cutScatterGrid.TotalValueLayer),2),"%"
         )
    fig_slices.canvas.draw_idle()
    
    

interact(update);

In [ ]:
#classic method
cutScatterGrid = scattergrid.cutDepthLayer(iz1+(iz2-iz1)/2,(iz2-iz1))
cutScatterGrid = scattergrid.cutDepthLayer(ld,ls)
trueval = targets.cutDepthLayer(*cutScatterGrid.ExtentZ).sum()
print('trueval: ',round(trueval,2))
print('classic     val:',round(cutScatterGrid.TotalValue,2))
print('new         val:',round(cutScatterGrid.TotalValueLayer,2))
print('new layer % val:',round(100*(1-trueval/cutScatterGrid.TotalValueLayer),2))
print('new %       val:',round(100*(1-trueval/cutScatterGrid.TotalValue),2))

In [ ]:

X_,Y_,Z_ = np.meshgrid(
    np.arange(scattergrid.MinX,scattergrid.MaxX,scattergrid.ResX),
    np.arange(scattergrid.MinY,scattergrid.MaxY,scattergrid.ResY),
    np.arange(scattergrid.MinZ,scattergrid.MaxZ,scattergrid.ResZ))

#T = np.exp(-X**2 - Y**2 - Z**2)


X=[]
Y=[]
Z=[]
V = []

   
if True:
    image = get_3DImage(scattergrid)
    gridder = scattergrid.getGridder()
    
    im_max = np.nanmax(image)
    
    min_val = im_max-70
    min_val = cmin+0.1
    print('im_max:',im_max)
    print('min_val:',min_val)

    for xi in range(image.shape[0]):
        for yi in range(image.shape[1]):
            for zi in range(image.shape[2]):
                if True:
                    if image[xi][yi][zi] < min_val:   
                        continue
                if False:
                    if image[xi][yi][zi] < -50:   
                        continue
                        
                if not np.isfinite(image[xi][yi][zi]):
                    continue
  
                if True:
                    if zi < zi1 or zi > zi2:
                        continue
                        
                X.append(gridder.get_x_value(xi))
                Y.append(gridder.get_y_value(yi))
                Z.append(gridder.get_z_value(zi))
                V.append(image[xi][yi][zi])
                
if True:      
    Xall=[]
    Yall=[]
    Zall=[]
    Vall = []
    for xi in range(image.shape[0]):
        for yi in range(image.shape[1]):
            for zi in range(image.shape[2]):            

                if not np.isfinite(image[xi][yi][zi]):
                    continue
                Xall.append(gridder.get_x_value(xi))
                Yall.append(gridder.get_y_value(yi))
                Zall.append(gridder.get_z_value(zi))
                Vall.append(image[xi][yi][zi])
            
    Xslx=[]
    Yslx=[]
    Zslx=[]
    Vslx = []
    selected_xi = max_resp_x[1]
    image_slice = get_3DImage(scattergrid,mindBVal=-70)[selected_xi,:,:]
    for yi in range(image_slice.shape[0]):
        for zi in range(image_slice.shape[1]):            
            
            if not np.isfinite(image[selected_xi][yi][zi]):
                continue
            Xslx.append(gridder.get_x_value(selected_xi))
            Yslx.append(gridder.get_y_value(yi))
            Zslx.append(gridder.get_z_value(zi))
            Vslx.append(image_slice[yi][zi])
    
    Xsly=[]
    Ysly=[]
    Zsly=[]
    Vsly = []
    selected_yi = max_resp_y[1]
    image_slice = get_3DImage(scattergrid,mindBVal=-70)[:,selected_yi,:]
    for xi in range(image_slice.shape[0]):
        for zi in range(image_slice.shape[1]):            
            
            if not np.isfinite(image[xi][selected_yi][zi]):
                continue
            Xsly.append(gridder.get_x_value(xi))
            Ysly.append(gridder.get_y_value(selected_yi))
            Zsly.append(gridder.get_z_value(zi))
            Vsly.append(image_slice[xi][zi])
            
    
    Xslz=[]
    Yslz=[]
    Zslz=[]
    Vslz = []
    selected_zi = max_resp_z[1]
    image_slice = get_3DImage(scattergrid,mindBVal=-70)[:,:,selected_zi]
    for xi in range(image_slice.shape[0]):
        for yi in range(image_slice.shape[1]):            
            
            if not np.isfinite(image[xi][yi][selected_zi]):
                continue
            Xslz.append(gridder.get_x_value(xi))
            Yslz.append(gridder.get_y_value(yi))
            Zslz.append(gridder.get_z_value(selected_zi))
            Vslz.append(image_slice[xi][zi])

alpha = np.array(V)
mia = np.nanmin(alpha)
maa = np.nanmax(alpha)
for n,a in enumerate(alpha):
    alpha[n] = (a - mia)/(maa-mia) + 0.01
alpha[alpha > 1] = 1



In [ ]:
import plotly.graph_objects as go
import numpy as np
import plotly.offline as pyo
import plotly.io as pio
pyo.init_notebook_mode(connected=False)

pio.renderers
import matplotlib


In [ ]:
cmap
plotly_cmap='ylgnbu_r'

In [ ]:
#cmin = -50
cmax = 0

try:
    del fig
except:
    pass
    
if True:
    fig = go.FigureWidget(data=[go.Scatter3d(
        showlegend=False,
        x=X,
        y=Y,
        z=Z,
        mode='markers',
        marker=dict(
            size=2,
            color=V,                # set color to an array/list of desired values
            #colorscale='Turbo',   # choose a colorscale
            #colorscale=[(0., "blue"), (1., "yellow")],
            colorscale=plotly_cmap,
            cmin = cmin,
            cmax = cmax,
            opacity=1,
            symbol='square',
            colorbar=dict(
                title="10log10(σ)",
                len=0.5
            )
        )
    )])
    
    
if True:
    fig.add_scatter3d(
        showlegend=False,
        x=Xslx,
        y=Yslx,
        z=Zslx,
        mode='markers',
        marker=dict(
            size=3,
            color=Vslx,                # set color to an array/list of desired values
            #color='red',
            colorscale=plotly_cmap,   # choose a colorscale
            cmin = cmin,
            cmax = cmax,
            opacity=0.1,
            symbol='square',
            #colorbar=dict(
            #    title="Colorbar",
            #    len=0.5
            #)
        )
    )
    
if False:
    fig.add_scatter3d(
        showlegend=False,
        x=Xsly,
        y=Ysly,
        z=Zsly,
        mode='markers',
        marker=dict(
            size=3,
            color=Vsly,                # set color to an array/list of desired values
            #color='blue',
            colorscale=plotly_cmap,   # choose a colorscale
            cmin = cmin,
            cmax = cmax,
            opacity=0.1,
            symbol='square',
            #colorbar=dict(
            #    title="Colorbar",
            #    len=0.5
            #)
        )
    )
    
if False:
    fig.add_scatter3d(
        showlegend=False,
        x=Xslz,
        y=Yslz,
        z=Zslz,
        mode='markers',
        marker=dict(
            size=3,
            #color=Vslz,                # set color to an array/list of desired values
            color='green',
            colorscale=plotly_cmap,   # choose a colorscale
            cmin = cmin,
            cmax = cmax,
            opacity=0.1,
            symbol='square',
            #colorbar=dict(
            #    title="Colorbar",
            #    len=0.5
            #)
        )
    )

# fig.add_scatter3d(
#     showlegend=False,
#     x=xbox,
#     y=ybox,
#     z=zbox,
#     mode='lines',
#     line=dict(color='blue', width=4)
# )

fig.add_scatter3d(
    showlegend=False,
    x=xbox_v12,
    y=ybox_v12,
    z=zbox_v1,
    mode='lines',
    line=dict(color='blue', width=4)
)
fig.add_scatter3d(
    showlegend=False,
    x=xbox_v12,
    y=ybox_v12,
    z=zbox_v2,
    mode='lines',
    line=dict(color='blue', width=4)
)
fig.add_scatter3d(
    showlegend=False,
    x=xbox_v3,
    y=ybox_v3,
    z=zbox_v3,
    mode='lines',
    line=dict(color='blue', width=4)
)


# fig.add_scatter3d(
#     showlegend=False,
#     x=xmbes,
#     y=ymbes,
#     z=zmbes,
#     mode='lines',
#     line=dict(color='red', width=4)
# )
fig.add_scatter3d(
    showlegend=False,
    x=xpath,
    y=ypath,
    z=zpath,
    #mode='lines',
    marker=dict(
        size=1,
        color='black',                
        opacity=1,
        symbol='square'
    ),
    line=dict(color='black', width=1)
)


fig.add_scatter3d(
        showlegend=False,
        x=setup.Simulation.Targets.x,
        y=setup.Simulation.Targets.y,
        z=setup.Simulation.Targets.z,
        mode='markers',
        marker=dict(
            size=0.3,
            color='black',                # set color to an array/list of desired values
            colorscale=plotly_cmap,   # choose a colorscale
            cmin = cmin,
            cmax = cmax,
            opacity=1,
            #symbol='square',
        )
    )

fig.update_layout(
    scene=dict(
        xaxis=dict(type="linear"),
        yaxis=dict(type="linear"),
        zaxis=dict(type="linear"),
        annotations=[
        dict(
            showarrow=True,
            x=setup.Simulation.Targets.x[0],
            y=setup.Simulation.Targets.y[0],
            z=setup.Simulation.Targets.z[0],
            #z=90,
            text="bubble stream position",
            xanchor="auto",
            ax=-30,
            ay=-55,
            arrowwidth=4,
            arrowcolor='black',
            opacity=1,
            xshift = -4,
            yshift = 4,
            #bgcolor='red',
            font=dict(
                color='black',
                size=18
            )
        )
        ]
    )
)



# tight layout
fig.update_layout(width=800,height =800)
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
#fig.update_scenes(zaxis_autorange="reversed", xaxis_autorange = 'reversed')

# fig.update_layout(
#     scene = dict(
#         xaxis = dict(range=[min(xbox),max(xbox)],),
#         yaxis = dict(range=[min(ymbes),max(ymbes)],),
#         zaxis = dict(range=[min(zmbes),max(zmbes)],),))

fig.update_layout(scene = dict(
                    xaxis_title='x / alongtack',
                    yaxis_title='y / acrosstrack',
                    zaxis_title='z / depth'))

fig.update_scenes(yaxis_autorange="reversed", zaxis_autorange = 'reversed')

fig

In [ ]:
print("end")